In [ ]:
def solve_overlapped(tau, customers, depot, Q, route_size):
    
    N = len(customers) # Number of customers
    M = int(N/route_size) # Number of trucks (assume N divisible by route_size)
    num_trips = 0
    
    # Create set of nodes with both depots and customers
    nodes = {}
    nodes.update(depots)
    nodes.update(customers)
    
    # Calculate pairwise distance matrix
    distances = calc_dist_matrix(nodes)

    # Get primary routes
    primary_routes = get_primary_routes(customers, tau, route_size)
    
    # Focus on customers with postive demand
    nonzero_routes = dict([(j, Route(sequence = dict([(i,primary_routes[j].sequence[i]) for i in primary_routes[j].sequence
                                                      if primary_routes[j].sequence[i].d > 0]), ID = j))
                           for j in primary_routes])

    ##### OVERLAPPING STRATEGY #####
    
    sarray = np.zeros(M, dtype=int) # Position of truck's starting customer
    warray = np.zeros(M, dtype=int) # Truck workload for primary customers
    earray = np.zeros(M, dtype=int) # Truck's surplus capacity upon finishing primary route
    darray = np.zeros(M, dtype=int) # The amount of demand for customer s_j that truck j serves       
    farray = np.zeros(M, dtype=int) # Final customer for each truck
    
    for j in range(M):
        my_route = nonzero_routes[j+1]
        if my_route.n > 0:
            if j == 0: # First route
                sarray[j] = min(i for i in tau if tau[i].d > 0)
                darray[j] = tau[sarray[j]].d
                warray[j] = my_route.d
                earray[j] = np.ceil(warray[j]/Q) * Q - warray[j]
            else: # All others
                sarray[j], darray[j] = get_split_info(my_route.sequence, earray[j-1])
                warray[j] = max(0, my_route.d - earray[j-2])
                earray[j] = np.ceil(warray[j-1]/Q) * Q - warray[j-1]
            
        # Determine previous vehicles's final customer
        #if j == M-1 and sarray[M-1] != 0: # If last truck starts route
         #   farray[M-1] = max(i for i in tau if tau[i].d > 0)
        if M == 1:
            farray[0] = max(i for i in nonzero_routes[j+1].sequence if tau[i].d > 0)
        elif j>=1: # Truck 2 to M-1
            if my_route.sequence == {}: # Truck j has no customers so truck j-1 covers only its own customers
                farray[j-1] = max(i for i in nonzero_routes[j+1].sequence if tau[i].d > 0)
                print('A')
            elif sarray[j] == 0: # Truck j-1 covers all of truck j's customers
                farray[j-1] = max(i for i in my_route.sequence if tau[i].d > 0)
                print('B')
            elif darray[j-1] == tau[sarray[j-1]].d: # Truck j covers all demand of customer s so truck j-1 ends with the preceding non-zero customer
                farray[j-1] = max(i for i in tau if i < sarray[j] if tau[i].d > 0) 
                print('C')
            else: # Trucks j-1 and j split customer s
                farray[j-1] = sarray[j] 
                print('D')
        
    ##### TRANSPORTATION COST #####
    circular, radial, num_trips = 0, 0, 0
    for j in range(M):
        print(sarray)
        print(farray)
        subseq = dict([(i, tau[i]) for i in nonzero_routes[j+1].sequence if i >= sarray[j] and i <= farray[j]])
        endpoints = get_radial_customers(subseq, warray[j], Q) # List of customers
        print(endpoints)
        # Number of trips
        assert len(endpoints) % 2 == 0, "Assertion Error: There is not an even number of endpoints."
        num_trips += len(endpoints)/2
        
        # Radial cost
        radial = sum(distances[my_depot.ID, c.ID] for c in endpoints)
        
        # Circular cost
        edges = get_routing_solution(nonzero_routes[j+1].sequence)
        circular += sum(distances[e] for e in edges)
        
    return circular, radial, circular+radial, num_trips
            